### Q1 Running Ollama with Docker

Running the following command first to save the image in a local folder:

In [ ]:
docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama

Once the image is pulled, use the following code to check the version:

In [ ]:
docker exec -it ollama bash
ollama -v

__Answer 1:__ ollama version is 0.1.48

### Q2 Downloading an LLM

Execute the following commands from bash to pull gemma2.


In [ ]:
ollama pull gemma:2b

Output

In [ ]:
pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 

__Answer 2:__ Content of the file models/manifests/registry.ollama.ai/library/gemma:

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}

### Q3 Running the LLM

Running a query in LLM: 10*10

__Answer 3:__

In [ ]:
Sure, here's the answer to your question:

10 * 10 = 100.

### Q4 Donwloading the weights

The size of ollama_files/models is:

In [ ]:
du -h
1.6G	./blobs
4.0K	./manifests/registry.ollama.ai/library/gemma
4.0K	./manifests/registry.ollama.ai/library
4.0K	./manifests/registry.ollama.ai
4.0K	./manifests
1.6G

__Answer 4:__ 1.7G

### Q5 Adding the weights

Dockerfile:

In [ ]:
FROM ollama/ollama

COPY ollama_files /root/.ollama

__Answer 5:__ ollama_files /root/.ollama

### Q6 Serving it

Building the image:

In [ ]:
docker build -t ollama-gemma2b .
[+] Building 72.9s (7/7) FINISHED                          docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 89B                                        0.0s
 => [internal] load metadata for docker.io/ollama/ollama:latest            0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build context                                         38.3s
 => => transferring context: 1.68GB                                       38.3s
 => [1/2] FROM docker.io/ollama/ollama:latest                              0.1s
 => [2/2] COPY ollama_files /root/.ollama                                 19.4s
 => exporting to image                                                    14.8s
 => => exporting layers                                                   14.8s
 => => writing image sha256:f45dac79ab199093e54dad1f87e4a686cea78c8e177b8  0.0s
 => => naming to docker.io/library/ollama-gemma2b                          0.0s

View build details: docker-desktop://dashboard/build/desktop-linux/desktop-linux/opluj70hzskxl56688ri2f0ow

Connecting to the image using OpenAI:

In [ ]:
# Connecting to OpenAI
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

# Defining the function with temperature 0
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content

# Executing the prompt
prompt = "What's the formula for energy?"
answer = llm(prompt)
print(answer)

In [ ]:
import os
from huggingface_hub import login

login(token=os.environ['HF_TOKEN'])

In [ ]:
# Counting the number of completion tokens using AutoTokenizer
# Import AutoTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

# Defining input sequence tokens
input_ids = tokenizer(answer, return_tensors="pt")

# Calculating the number of tokens
outputs = model.generate(**input_ids)
completion_tokens = len(outputs[0]) - len(input_ids['input_ids'][0])
print(f"Number of completion tokens: {completion_tokens}")

In [ ]:
# Counting the number of completion tokens using GemmaTokenizer
# Import GemmaTokenizer
from transformers import GemmaTokenizer, GemmaForCausalLM

# Set the tokenizer and the model
tokenizer = GemmaTokenizer.from_pretrained("google/gemma-2b")
model = GemmaForCausalLM.from_pretrained("google/gemma-2b")

# Defining input sequence tokens
input_ids = tokenizer(answer, return_tensors="pt")

# Calculating the number of tokens
outputs = model.generate(**input_ids)
completion_tokens = len(outputs[0]) - len(input_ids['input_ids'][0])
print(f"Number of completion tokens: {completion_tokens}")